In [2]:
import numpy as np 
import pandas as pd 

import os
# Basic library
import numpy as np 
import pandas as pd 
import tensorflow as tf
import keras

# Data preprocessing
import cv2# Open cv
from sklearn.model_selection import train_test_split

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import pickle


In [3]:
# model= tf.keras.models.load_model('/home/ahmed/queens_project/final_project/semantic_segmentation.h5')

In [6]:
test_drive_df= pd.read_csv('/home/ahmed/queens_project/final_project/full_data.csv', index_col=0) 

In [9]:
test_drive_df

,Rec_Lat,Rec_Long,RSRP,serving_Lat,serving_Long,distance,Bearing_angle,PL
0,30.01471,31.45804,-66.5625,30.010772,31.456667,0.456179,-163.196082,121.064588
1,30.01472,31.45804,-66.5625,30.010772,31.456667,0.457239,-163.236257,121.099297
2,30.01472,31.45805,-66.5625,30.010772,31.456667,0.457520,-163.121105,121.108460
3,30.01472,31.45806,-66.8125,30.010772,31.456667,0.457802,-163.006094,121.117678
4,30.01472,31.45807,-67.3125,30.010772,31.456667,0.458086,-162.891224,121.126951
...,...,...,...,...,...,...,...,...
86899,30.00200,31.46485,-76.8750,30.002019,31.461689,0.305004,-89.593181,115.049333
86900,30.00200,31.46486,-76.8750,30.002019,31.461689,0.305969,-89.594459,115.096526
86901,30.00200,31.46487,-76.6875,30.002019,31.461689,0.306934,-89.595728,115.143570
86902,30.00200,31.46488,-79.3750,30.002019,31.461689,0.307899,-89.596990,115.190467


In [7]:
path = '/home/ahmed/queens_project/final_project/Satellite_Image/Satellite_Images' # current directory
filenames = [os.path.abspath(os.path.join(path, x)) for x in os.listdir(path) if '.jpg' in x]
# filenames

In [8]:
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


filenames.sort(key=natural_keys)

images_df= pd.DataFrame()
images_df["images"] = filenames

images_df

,images
0,/home/ahmed/queens_project/final_project/Satel...
1,/home/ahmed/queens_project/final_project/Satel...
2,/home/ahmed/queens_project/final_project/Satel...
3,/home/ahmed/queens_project/final_project/Satel...
4,/home/ahmed/queens_project/final_project/Satel...
...,...
86899,/home/ahmed/queens_project/final_project/Satel...
86900,/home/ahmed/queens_project/final_project/Satel...
86901,/home/ahmed/queens_project/final_project/Satel...
86902,/home/ahmed/queens_project/final_project/Satel...


In [10]:
test_drive_df["images"]=np.nan
for i in test_drive_df.index:
  test_drive_df["images"][i]=images_df["images"][i]

In [11]:
import cv2
from tqdm import tqdm
from rich.progress import track

output=[]
for i in track(range(len(images_df))):
  src = cv2.imread(str(images_df["images"][i]), cv2.IMREAD_UNCHANGED)

  width=256
  height=256

  # dsize
  dsize = (width, height)

  # resize image
  output.append(cv2.resize(src, dsize))

  # cv2.imwrite(str(images_df["images"][i]),output[i]) 

Output()

In [13]:
resized_img=[]
for i in test_drive_df.index:
  resized_img.append(output[i])
  resized_images = np.array(resized_img)

KeyboardInterrupt: 

In [4]:
new_model = tf.keras.models.load_model('/home/ahmed/queens_project/final_project/Best_semantic_segmentation.h5' , compile=False)

2022-10-21 08:31:31.010515: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ahmed/.conda/envs/queens_env/lib/python3.8/site-packages/cv2/../../lib64:/usr/local/cuda-11.6/lib64:
2022-10-21 08:31:31.010548: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-21 08:31:31.010871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
T

In [5]:
new_model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_133 (Conv2D)            (None, 256, 256, 16  448         ['input_8[0][0]']                
                                )                                                                 
                                                                                                  
 dropout_63 (Dropout)           (None, 256, 256, 16  0           ['conv2d_133[0][0]']             
                                )                                                           

In [ ]:
y_pred= new_model.predict(resized_images)
